In [1]:
from astropy.io import ascii, fits
import astropy
import pylab as plt
%matplotlib inline
from astropy import wcs
from astropy.table import Table,Column,join,hstack,vstack
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import WMAP9 as cosmo
import pymoc
import glob
from time import sleep
import os
from astropy.nddata.utils import Cutout2D
import herschelhelp.cutouts_server as ctts

import numpy as np
import xidplus
from xidplus import moc_routines
import pickle
import xidplus.catalogue as cat

import sys
from herschelhelp_internal.utils import inMoc,flux_to_mag
from xidplus.stan_fit import SPIRE

import aplpy
import seaborn as sns
#sns.set(color_codes=True)
import pandas as pd
#sns.set_style("white")
import xidplus.posterior_maps as postmaps
from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp import image_plotting
import pyvo as vo
import glob

import copy

In [7]:
lofar_v1 = Table.read('data/data_release/final_cross_match_catalogue-v0.1.fits')
print(len(lofar_v1))
lofar_v1[0]

31748


Source_Name,RA,DEC,E_RA,E_DEC,Total_flux,E_Total_flux,Peak_flux,E_Peak_flux,S_Code,Maj,Min,PA,E_Maj,E_Min,E_PA,DC_Maj,DC_Min,DC_PA,FLAG_WORKFLOW,Prefilter,lr_fin,optRA,optDec,LGZ_Size,LGZ_Width,LGZ_PA,Assoc,Assoc_Qual,Art_prob,Blend_prob,Hostbroken_prob,Imagemissing_prob,Zoom_prob,Created,Position_from,Renamed_from,FLAG_OVERLAP_RADIO,flag_clean_radio,NUMBER,X_IMAGE,Y_IMAGE,ALPHA_J2000,DELTA_J2000,flag_clean,NUMBER_OPTICAL,NUMBER_SPITZER,FLAG_OVERLAP,Separation,help_id,RA_HELP,DEC_HELP,EBV,Z_SPEC,CLASS_STAR,u_flux,u_fluxerr,g_flux,g_fluxerr,r_flux,r_fluxerr,i_flux,i_fluxerr,z_flux,z_fluxerr,y_flux,y_fluxerr,g_hsc_flux,g_hsc_fluxerr,r_hsc_flux,r_hsc_fluxerr,i_hsc_flux,i_hsc_fluxerr,z_hsc_flux,z_hsc_fluxerr,y_hsc_flux,y_hsc_fluxerr,nb921_hsc_flux,nb921_hsc_fluxerr,J_flux,J_fluxerr,K_flux,K_fluxerr,ch1_servs_flux,ch1_servs_fluxerr,ch2_servs_flux,ch2_servs_fluxerr,ch1_swire_flux,ch1_swire_fluxerr,ch2_swire_flux,ch2_swire_fluxerr,ch3_swire_flux,ch3_swire_fluxerr,ch4_swire_flux,ch4_swire_fluxerr,F_MIPS_24,FErr_MIPS_24_u,FErr_MIPS_24_l,Bkg_MIPS_24,Sig_conf_MIPS_24,Rhat_MIPS_24,n_eff_MIPS_24,Pval_res_24,flag_mips_24,F_PACS_100,FErr_PACS_100_u,FErr_PACS_100_l,F_PACS_160,FErr_PACS_160_u,FErr_PACS_160_l,Bkg_PACS_100,Bkg_PACS_160,Sig_conf_PACS_100,Sig_conf_PACS_160,Rhat_PACS_100,Rhat_PACS_160,n_eff_PACS_100,n_eff_PACS_160,Pval_res_100,Pval_res_160,flag_PACS_100,flag_PACS_160,F_SPIRE_250,FErr_SPIRE_250_u,FErr_SPIRE_250_l,F_SPIRE_350,FErr_SPIRE_350_u,FErr_SPIRE_350_l,F_SPIRE_500,FErr_SPIRE_500_u,FErr_SPIRE_500_l,Bkg_SPIRE_250,Bkg_SPIRE_350,Bkg_SPIRE_500,Sig_conf_SPIRE_250,Sig_conf_SPIRE_350,Sig_conf_SPIRE_500,Rhat_SPIRE_250,Rhat_SPIRE_350,Rhat_SPIRE_500,n_eff_SPIRE_250,n_eff_SPIRE_500,n_eff_SPIRE_350,Pval_res_250,Pval_res_350,Pval_res_500,flag_spire_250,flag_spire_350,flag_spire_500,AGN,optAGN,IRAGN,XrayAGN,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,nfilt_eazy,nfilt_atlas,nfilt_ananna,chi_r_best,chi_r_stellar,stellar_type,hp_idx_11,hp_depth_cluster
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,
bytes22,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,bytes22,bytes17,bytes22,int64,int64,int64,float32,float32,float64,float64,int64,int64,int32,int64,float64,bytes27,float64,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,bool,bool,bool,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,bytes6,int64,int32
ILTJ155957.58+550052.4,239.98990167026162,55.01456045936245,5.366954914924915e-05,8.08706707123675e-05,0.0003958345213565919,5.4160526835678236e-05,0.0003247963953282784,2.715224507873135e-05,S,0.002071920400992901,0.0016345290335764496,166.30848152559156,0.00019279093100176814,0.00012276320016402095,13.835288085283342,0.0,0.0,0.0,1,0,329.1186755105229,239.98983453474398,55.01474018731156,nan,nan,nan,0,nan,nan,nan,nan,nan,nan,Create initial sources,LR,,7,1,1018637,73464.984,38011.69,239.98983453474398,55.01474018731156,1,1180605,--,7

In [6]:
lofar_v3 = Table.read('data/data_release/final_cross_match_catalogue-v0.3.fits')
print(len(lofar_v3))
lofar_v3[0]

31650


Source_Name,RA,DEC,E_RA,E_DEC,Total_flux,E_Total_flux,Peak_flux,E_Peak_flux,S_Code,Maj,Min,PA,E_Maj,E_Min,E_PA,DC_Maj,DC_Min,DC_PA,FLAG_WORKFLOW,Prefilter,NoID,lr_fin,optRA,optDec,LGZ_Size,LGZ_Width,LGZ_PA,Assoc,Assoc_Qual,Art_prob,Blend_prob,Hostbroken_prob,Imagemissing_prob,Zoom_prob,Created,Position_from,Renamed_from,FLAG_OVERLAP_RADIO,flag_clean_radio,NUMBER,X_IMAGE,Y_IMAGE,ALPHA_J2000,DELTA_J2000,flag_clean,NUMBER_OPTICAL,NUMBER_SPITZER,FLAG_OVERLAP,Separation,help_id,RA_HELP,DEC_HELP,EBV,Z_SPEC,CLASS_STAR,u_flux,u_fluxerr,g_flux,g_fluxerr,r_flux,r_fluxerr,i_flux,i_fluxerr,z_flux,z_fluxerr,y_flux,y_fluxerr,g_hsc_flux,g_hsc_fluxerr,r_hsc_flux,r_hsc_fluxerr,i_hsc_flux,i_hsc_fluxerr,z_hsc_flux,z_hsc_fluxerr,y_hsc_flux,y_hsc_fluxerr,nb921_hsc_flux,nb921_hsc_fluxerr,J_flux,J_fluxerr,K_flux,K_fluxerr,ch1_servs_flux,ch1_servs_fluxerr,ch2_servs_flux,ch2_servs_fluxerr,ch1_swire_flux,ch1_swire_fluxerr,ch2_swire_flux,ch2_swire_fluxerr,ch3_swire_flux,ch3_swire_fluxerr,ch4_swire_flux,ch4_swire_fluxerr,F_MIPS_24,FErr_MIPS_24_u,FErr_MIPS_24_l,Bkg_MIPS_24,Sig_conf_MIPS_24,Rhat_MIPS_24,n_eff_MIPS_24,Pval_res_24,flag_mips_24,F_PACS_100,FErr_PACS_100_u,FErr_PACS_100_l,F_PACS_160,FErr_PACS_160_u,FErr_PACS_160_l,Bkg_PACS_100,Bkg_PACS_160,Sig_conf_PACS_100,Sig_conf_PACS_160,Rhat_PACS_100,Rhat_PACS_160,n_eff_PACS_100,n_eff_PACS_160,Pval_res_100,Pval_res_160,flag_PACS_100,flag_PACS_160,F_SPIRE_250,FErr_SPIRE_250_u,FErr_SPIRE_250_l,F_SPIRE_350,FErr_SPIRE_350_u,FErr_SPIRE_350_l,F_SPIRE_500,FErr_SPIRE_500_u,FErr_SPIRE_500_l,Bkg_SPIRE_250,Bkg_SPIRE_350,Bkg_SPIRE_500,Sig_conf_SPIRE_250,Sig_conf_SPIRE_350,Sig_conf_SPIRE_500,Rhat_SPIRE_250,Rhat_SPIRE_350,Rhat_SPIRE_500,n_eff_SPIRE_250,n_eff_SPIRE_500,n_eff_SPIRE_350,Pval_res_250,Pval_res_350,Pval_res_500,flag_spire_250,flag_spire_350,flag_spire_500,AGN,optAGN,IRAGN,XrayAGN,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,nfilt_eazy,nfilt_atlas,nfilt_ananna,chi_r_best,chi_r_stellar,stellar_type,hp_idx_11,hp_depth_cluster
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,
bytes22,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,bytes22,bytes17,bytes22,int64,int64,int64,float32,float32,float64,float64,int64,int64,int32,int64,float64,bytes27,float64,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,bool,bool,bool,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,bytes6,int64,int32
ILTJ155957.58+550052.4,239.98990167026162,55.01456045936245,5.366954914924915e-05,8.08706707123675e-05,0.0003958345213565919,5.4160526835678236e-05,0.0003247963953282784,2.715224507873135e-05,S,0.002071920400992901,0.0016345290335764496,166.30848152559156,0.00019279093100176814,0.00012276320016402095,13.835288085283342,0.0,0.0,0.0,1,0,0,329.1186755105229,239.98983453474398,55.01474018731156,nan,nan,nan,0,nan,nan,nan,nan,nan,nan,Create initial sources,LR,,7,1,1018637,73464.984,38011.69,239.98983453474398,55.01474018731156,

In [15]:
lofar_fir = Table.read('data/data_release/XID+_lofar_ELAIS-N1.fits')
lofar_fir.rename_column('Source_Name_1','Source_Name')
print(len(lofar_fir))
lofar_fir[0]

8568


Source_Name,RA,DEC_1,F_SPIRE_250_1,FErr_SPIRE_250_u_1,FErr_SPIRE_250_l_1,F_SPIRE_350_1,FErr_SPIRE_350_u_1,FErr_SPIRE_350_l_1,F_SPIRE_500_1,FErr_SPIRE_500_u_1,FErr_SPIRE_500_l_1,Bkg_SPIRE_250_1,Bkg_SPIRE_350_1,Bkg_SPIRE_500_1,Sig_conf_SPIRE_250_1,Sig_conf_SPIRE_350_1,Sig_conf_SPIRE_500_1,Rhat_SPIRE_250_1,Rhat_SPIRE_350_1,Rhat_SPIRE_500_1,n_eff_SPIRE_250_1,n_eff_SPIRE_500_1,n_eff_SPIRE_350_1,Pval_res_250_1,Pval_res_350_1,Pval_res_500_1,flag_spire_250_1,flag_spire_350_1,flag_spire_500_1,XID+_rerun_SPIRE,RA_2,Dec_2,F_PACS_100_1,FErr_PACS_100_u_1,FErr_PACS_100_l_1,F_PACS_160_1,FErr_PACS_160_u_1,FErr_PACS_160_l_1,Bkg_PACS_100_1,Bkg_PACS_160_1,Sig_conf_PACS_100_1,Sig_conf_PACS_160_1,Rhat_PACS_100_1,Rhat_PACS_160_1,n_eff_PACS_100_1,n_eff_PACS_160_1,Pval_res_100_1,Pval_res_160_1,flag_pacs_100,flag_pacs_160,RA_3,Dec_3,F_MIPS_24_1,FErr_MIPS_24_u_1,FErr_MIPS_24_l_1,Bkg_MIPS_24_1,Sig_conf_MIPS_24_1,Rhat_MIPS_24_1,n_eff_MIPS_24_1,Pval_res_24_1,flag_mips_24_1,XID+_rerun_mips,Source_Name_2,DEC_2,E_RA,E_DEC,Total_flux,E_Total_flux,Peak_flux,E_Peak_flux,S_Code,Maj,Min,PA,E_Maj,E_Min,E_PA,DC_Maj,DC_Min,DC_PA,FLAG_WORKFLOW,Prefilter,NoID,lr_fin,optRA,optDec,LGZ_Size,LGZ_Width,LGZ_PA,Assoc,Assoc_Qual,Art_prob,Blend_prob,Hostbroken_prob,Imagemissing_prob,Zoom_prob,Created,Position_from,Renamed_from,FLAG_OVERLAP_RADIO,flag_clean_radio,NUMBER,X_IMAGE,Y_IMAGE,ALPHA_J2000,DELTA_J2000,flag_clean,NUMBER_OPTICAL,NUMBER_SPITZER,FLAG_OVERLAP,Separation,help_id,RA_HELP,DEC_HELP,EBV,Z_SPEC,CLASS_STAR,u_flux,u_fluxerr,g_flux,g_fluxerr,r_flux,r_fluxerr,i_flux,i_fluxerr,z_flux,z_fluxerr,y_flux,y_fluxerr,g_hsc_flux,g_hsc_fluxerr,r_hsc_flux,r_hsc_fluxerr,i_hsc_flux,i_hsc_fluxerr,z_hsc_flux,z_hsc_fluxerr,y_hsc_flux,y_hsc_fluxerr,nb921_hsc_flux,nb921_hsc_fluxerr,J_flux,J_fluxerr,K_flux,K_fluxerr,ch1_servs_flux,ch1_servs_fluxerr,ch2_servs_flux,ch2_servs_fluxerr,ch1_swire_flux,ch1_swire_fluxerr,ch2_swire_flux,ch2_swire_fluxerr,ch3_swire_flux,ch3_swire_fluxerr,ch4_swire_flux,ch4_swire_fluxerr,F_MIPS_24_2,FErr_MIPS_24_u_2,FErr_MIPS_24_l_2,Bkg_MIPS_24_2,Sig_conf_MIPS_24_2,Rhat_MIPS_24_2,n_eff_MIPS_24_2,Pval_res_24_2,flag_mips_24_2,F_PACS_100_2,FErr_PACS_100_u_2,FErr_PACS_100_l_2,F_PACS_160_2,FErr_PACS_160_u_2,FErr_PACS_160_l_2,Bkg_PACS_100_2,Bkg_PACS_160_2,Sig_conf_PACS_100_2,Sig_conf_PACS_160_2,Rhat_PACS_100_2,Rhat_PACS_160_2,n_eff_PACS_100_2,n_eff_PACS_160_2,Pval_res_100_2,Pval_res_160_2,flag_PACS_100,flag_PACS_160,F_SPIRE_250_2,FErr_SPIRE_250_u_2,FErr_SPIRE_250_l_2,F_SPIRE_350_2,FErr_SPIRE_350_u_2,FErr_SPIRE_350_l_2,F_SPIRE_500_2,FErr_SPIRE_500_u_2,FErr_SPIRE_500_l_2,Bkg_SPIRE_250_2,Bkg_SPIRE_350_2,Bkg_SPIRE_500_2,Sig_conf_SPIRE_250_2,Sig_conf_SPIRE_350_2,Sig_conf_SPIRE_500_2,Rhat_SPIRE_250_2,Rhat_SPIRE_350_2,Rhat_SPIRE_500_2,n_eff_SPIRE_250_2,n_eff_SPIRE_500_2,n_eff_SPIRE_350_2,Pval_res_250_2,Pval_res_350_2,Pval_res_500_2,flag_spire_250_2,flag_spire_350_2,flag_spire_500_2,AGN,optAGN,IRAGN,XrayAGN,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,nfilt_eazy,nfilt_atlas,nfilt_ananna,chi_r_best,chi_r_stellar,stellar_type,hp_idx_11,hp_depth_cluster
,degrees,degrees,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,,,,,,,,,,,,,,degrees,degrees,mJy,mJy,mJy,mJy,mJy,mJy,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,,,,,,,,,degrees,degrees,muJy,muJy,muJy,MJy / sr,MJy / sr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,
bytes27,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,bool,bool,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,bytes22,float64,

In [25]:
v1_v3 = join(lofar_v1,lofar_v3,keys='Source_Name',join_type='inner',table_names=['v1','v3'])
print(len(v1_v3))
v1_v3[0]

31592


Source_Name,RA_v1,DEC_v1,E_RA_v1,E_DEC_v1,Total_flux_v1,E_Total_flux_v1,Peak_flux_v1,E_Peak_flux_v1,S_Code_v1,Maj_v1,Min_v1,PA_v1,E_Maj_v1,E_Min_v1,E_PA_v1,DC_Maj_v1,DC_Min_v1,DC_PA_v1,FLAG_WORKFLOW_v1,Prefilter_v1,lr_fin_v1,optRA_v1,optDec_v1,LGZ_Size_v1,LGZ_Width_v1,LGZ_PA_v1,Assoc_v1,Assoc_Qual_v1,Art_prob_v1,Blend_prob_v1,Hostbroken_prob_v1,Imagemissing_prob_v1,Zoom_prob_v1,Created_v1,Position_from_v1,Renamed_from_v1,FLAG_OVERLAP_RADIO_v1,flag_clean_radio_v1,NUMBER_v1,X_IMAGE_v1,Y_IMAGE_v1,ALPHA_J2000_v1,DELTA_J2000_v1,flag_clean_v1,NUMBER_OPTICAL_v1,NUMBER_SPITZER_v1,FLAG_OVERLAP_v1,Separation_v1,help_id_v1,RA_HELP_v1,DEC_HELP_v1,EBV_v1,Z_SPEC_v1,CLASS_STAR_v1,u_flux_v1,u_fluxerr_v1,g_flux_v1,g_fluxerr_v1,r_flux_v1,r_fluxerr_v1,i_flux_v1,i_fluxerr_v1,z_flux_v1,z_fluxerr_v1,y_flux_v1,y_fluxerr_v1,g_hsc_flux_v1,g_hsc_fluxerr_v1,r_hsc_flux_v1,r_hsc_fluxerr_v1,i_hsc_flux_v1,i_hsc_fluxerr_v1,z_hsc_flux_v1,z_hsc_fluxerr_v1,y_hsc_flux_v1,y_hsc_fluxerr_v1,nb921_hsc_flux_v1,nb921_hsc_fluxerr_v1,J_flux_v1,J_fluxerr_v1,K_flux_v1,K_fluxerr_v1,ch1_servs_flux_v1,ch1_servs_fluxerr_v1,ch2_servs_flux_v1,ch2_servs_fluxerr_v1,ch1_swire_flux_v1,ch1_swire_fluxerr_v1,ch2_swire_flux_v1,ch2_swire_fluxerr_v1,ch3_swire_flux_v1,ch3_swire_fluxerr_v1,ch4_swire_flux_v1,ch4_swire_fluxerr_v1,F_MIPS_24_v1,FErr_MIPS_24_u_v1,FErr_MIPS_24_l_v1,Bkg_MIPS_24_v1,Sig_conf_MIPS_24_v1,Rhat_MIPS_24_v1,n_eff_MIPS_24_v1,Pval_res_24_v1,flag_mips_24_v1,F_PACS_100_v1,FErr_PACS_100_u_v1,FErr_PACS_100_l_v1,F_PACS_160_v1,FErr_PACS_160_u_v1,FErr_PACS_160_l_v1,Bkg_PACS_100_v1,Bkg_PACS_160_v1,Sig_conf_PACS_100_v1,Sig_conf_PACS_160_v1,Rhat_PACS_100_v1,Rhat_PACS_160_v1,n_eff_PACS_100_v1,n_eff_PACS_160_v1,Pval_res_100_v1,Pval_res_160_v1,flag_PACS_100_v1,flag_PACS_160_v1,F_SPIRE_250_v1,FErr_SPIRE_250_u_v1,FErr_SPIRE_250_l_v1,F_SPIRE_350_v1,FErr_SPIRE_350_u_v1,FErr_SPIRE_350_l_v1,F_SPIRE_500_v1,FErr_SPIRE_500_u_v1,FErr_SPIRE_500_l_v1,Bkg_SPIRE_250_v1,Bkg_SPIRE_350_v1,Bkg_SPIRE_500_v1,Sig_conf_SPIRE_250_v1,Sig_conf_SPIRE_350_v1,Sig_conf_SPIRE_500_v1,Rhat_SPIRE_250_v1,Rhat_SPIRE_350_v1,Rhat_SPIRE_500_v1,n_eff_SPIRE_250_v1,n_eff_SPIRE_500_v1,n_eff_SPIRE_350_v1,Pval_res_250_v1,Pval_res_350_v1,Pval_res_500_v1,flag_spire_250_v1,flag_spire_350_v1,flag_spire_500_v1,AGN_v1,optAGN_v1,IRAGN_v1,XrayAGN_v1,z1_median_v1,z1_min_v1,z1_max_v1,z1_area_v1,z2_median_v1,z2_min_v1,z2_max_v1,z2_area_v1,nfilt_eazy_v1,nfilt_atlas_v1,nfilt_ananna_v1,chi_r_best_v1,chi_r_stellar_v1,stellar_type_v1,hp_idx_11_v1,hp_depth_cluster_v1,RA_v3,DEC_v3,E_RA_v3,E_DEC_v3,Total_flux_v3,E_Total_flux_v3,Peak_flux_v3,E_Peak_flux_v3,S_Code_v3,Maj_v3,Min_v3,PA_v3,E_Maj_v3,E_Min_v3,E_PA_v3,DC_Maj_v3,DC_Min_v3,DC_PA_v3,FLAG_WORKFLOW_v3,Prefilter_v3,NoID,lr_fin_v3,optRA_v3,optDec_v3,LGZ_Size_v3,LGZ_Width_v3,LGZ_PA_v3,Assoc_v3,Assoc_Qual_v3,Art_prob_v3,Blend_prob_v3,Hostbroken_prob_v3,Imagemissing_prob_v3,Zoom_prob_v3,Created_v3,Position_from_v3,Renamed_from_v3,FLAG_OVERLAP_RADIO_v3,flag_clean_radio_v3,NUMBER_v3,X_IMAGE_v3,Y_IMAGE_v3,ALPHA_J2000_v3,DELTA_J2000_v3,flag_clean_v3,NUMBER_OPTICAL_v3,NUMBER_SPITZER_v3,FLAG_OVERLAP_v3,Separation_v3,help_id_v3,RA_HELP_v3,DEC_HELP_v3,EBV_v3,Z_SPEC_v3,CLASS_STAR_v3,u_flux_v3,u_fluxerr_v3,g_flux_v3,g_fluxerr_v3,r_flux_v3,r_fluxerr_v3,i_flux_v3,i_fluxerr_v3,z_flux_v3,z_fluxerr_v3,y_flux_v3,y_fluxerr_v3,g_hsc_flux_v3,g_hsc_fluxerr_v3,r_hsc_flux_v3,r_hsc_fluxerr_v3,i_hsc_flux_v3,i_hsc_fluxerr_v3,z_hsc_flux_v3,z_hsc_fluxerr_v3,y_hsc_flux_v3,y_hsc_fluxerr_v3,nb921_hsc_flux_v3,nb921_hsc_fluxerr_v3,J_flux_v3,J_fluxerr_v3,K_flux_v3,K_fluxerr_v3,ch1_servs_flux_v3,ch1_servs_fluxerr_v3,ch2_servs_flux_v3,ch2_servs_fluxerr_v3,ch1_swire_flux_v3,ch1_swire_fluxerr_v3,ch2_swire_flux_v3,ch2_swire_fluxerr_v3,ch3_swire_flux_v3,ch3_swire_fluxerr_v3,ch4_swire_flux_v3,ch4_swire_fluxerr_v3,F_MIPS_24_v3,FErr_MIPS_24_u_v3,FErr_MIPS_24_l_v3,Bkg_MIPS_24_v3,Sig_conf_MIPS_24_v3,Rhat_MIPS_24_v3,n_eff_MIPS_24_v3,Pval_res_24_v3,flag_mips_24_v3,F_PACS_100_v3,FErr_PACS_100_u_v3,FErr_PACS_100_l_v3,F_PACS_160_v3,FErr_PACS_160_

In [44]:
fir_v3 = join(lofar_v3,lofar_fir,join_type='inner',keys='Source_Name',table_names=['v3','fir'])
print(len(fir_v3))
fir_v3[0]

8428


Source_Name,RA_v3,DEC,E_RA_v3,E_DEC_v3,Total_flux_v3,E_Total_flux_v3,Peak_flux_v3,E_Peak_flux_v3,S_Code_v3,Maj_v3,Min_v3,PA_v3,E_Maj_v3,E_Min_v3,E_PA_v3,DC_Maj_v3,DC_Min_v3,DC_PA_v3,FLAG_WORKFLOW_v3,Prefilter_v3,NoID_v3,lr_fin_v3,optRA_v3,optDec_v3,LGZ_Size_v3,LGZ_Width_v3,LGZ_PA_v3,Assoc_v3,Assoc_Qual_v3,Art_prob_v3,Blend_prob_v3,Hostbroken_prob_v3,Imagemissing_prob_v3,Zoom_prob_v3,Created_v3,Position_from_v3,Renamed_from_v3,FLAG_OVERLAP_RADIO_v3,flag_clean_radio_v3,NUMBER_v3,X_IMAGE_v3,Y_IMAGE_v3,ALPHA_J2000_v3,DELTA_J2000_v3,flag_clean_v3,NUMBER_OPTICAL_v3,NUMBER_SPITZER_v3,FLAG_OVERLAP_v3,Separation_v3,help_id_v3,RA_HELP_v3,DEC_HELP_v3,EBV_v3,Z_SPEC_v3,CLASS_STAR_v3,u_flux_v3,u_fluxerr_v3,g_flux_v3,g_fluxerr_v3,r_flux_v3,r_fluxerr_v3,i_flux_v3,i_fluxerr_v3,z_flux_v3,z_fluxerr_v3,y_flux_v3,y_fluxerr_v3,g_hsc_flux_v3,g_hsc_fluxerr_v3,r_hsc_flux_v3,r_hsc_fluxerr_v3,i_hsc_flux_v3,i_hsc_fluxerr_v3,z_hsc_flux_v3,z_hsc_fluxerr_v3,y_hsc_flux_v3,y_hsc_fluxerr_v3,nb921_hsc_flux_v3,nb921_hsc_fluxerr_v3,J_flux_v3,J_fluxerr_v3,K_flux_v3,K_fluxerr_v3,ch1_servs_flux_v3,ch1_servs_fluxerr_v3,ch2_servs_flux_v3,ch2_servs_fluxerr_v3,ch1_swire_flux_v3,ch1_swire_fluxerr_v3,ch2_swire_flux_v3,ch2_swire_fluxerr_v3,ch3_swire_flux_v3,ch3_swire_fluxerr_v3,ch4_swire_flux_v3,ch4_swire_fluxerr_v3,F_MIPS_24,FErr_MIPS_24_u,FErr_MIPS_24_l,Bkg_MIPS_24,Sig_conf_MIPS_24,Rhat_MIPS_24,n_eff_MIPS_24,Pval_res_24,flag_mips_24,F_PACS_100,FErr_PACS_100_u,FErr_PACS_100_l,F_PACS_160,FErr_PACS_160_u,FErr_PACS_160_l,Bkg_PACS_100,Bkg_PACS_160,Sig_conf_PACS_100,Sig_conf_PACS_160,Rhat_PACS_100,Rhat_PACS_160,n_eff_PACS_100,n_eff_PACS_160,Pval_res_100,Pval_res_160,flag_PACS_100_v3,flag_PACS_160_v3,F_SPIRE_250,FErr_SPIRE_250_u,FErr_SPIRE_250_l,F_SPIRE_350,FErr_SPIRE_350_u,FErr_SPIRE_350_l,F_SPIRE_500,FErr_SPIRE_500_u,FErr_SPIRE_500_l,Bkg_SPIRE_250,Bkg_SPIRE_350,Bkg_SPIRE_500,Sig_conf_SPIRE_250,Sig_conf_SPIRE_350,Sig_conf_SPIRE_500,Rhat_SPIRE_250,Rhat_SPIRE_350,Rhat_SPIRE_500,n_eff_SPIRE_250,n_eff_SPIRE_500,n_eff_SPIRE_350,Pval_res_250,Pval_res_350,Pval_res_500,flag_spire_250,flag_spire_350,flag_spire_500,AGN_v3,optAGN_v3,IRAGN_v3,XrayAGN_v3,z1_median_v3,z1_min_v3,z1_max_v3,z1_area_v3,z2_median_v3,z2_min_v3,z2_max_v3,z2_area_v3,nfilt_eazy_v3,nfilt_atlas_v3,nfilt_ananna_v3,chi_r_best_v3,chi_r_stellar_v3,stellar_type_v3,hp_idx_11_v3,hp_depth_cluster_v3,RA_fir,DEC_1,F_SPIRE_250_1,FErr_SPIRE_250_u_1,FErr_SPIRE_250_l_1,F_SPIRE_350_1,FErr_SPIRE_350_u_1,FErr_SPIRE_350_l_1,F_SPIRE_500_1,FErr_SPIRE_500_u_1,FErr_SPIRE_500_l_1,Bkg_SPIRE_250_1,Bkg_SPIRE_350_1,Bkg_SPIRE_500_1,Sig_conf_SPIRE_250_1,Sig_conf_SPIRE_350_1,Sig_conf_SPIRE_500_1,Rhat_SPIRE_250_1,Rhat_SPIRE_350_1,Rhat_SPIRE_500_1,n_eff_SPIRE_250_1,n_eff_SPIRE_500_1,n_eff_SPIRE_350_1,Pval_res_250_1,Pval_res_350_1,Pval_res_500_1,flag_spire_250_1,flag_spire_350_1,flag_spire_500_1,XID+_rerun_SPIRE,RA_2,Dec_2,F_PACS_100_1,FErr_PACS_100_u_1,FErr_PACS_100_l_1,F_PACS_160_1,FErr_PACS_160_u_1,FErr_PACS_160_l_1,Bkg_PACS_100_1,Bkg_PACS_160_1,Sig_conf_PACS_100_1,Sig_conf_PACS_160_1,Rhat_PACS_100_1,Rhat_PACS_160_1,n_eff_PACS_100_1,n_eff_PACS_160_1,Pval_res_100_1,Pval_res_160_1,flag_pacs_100,flag_pacs_160,RA_3,Dec_3,F_MIPS_24_1,FErr_MIPS_24_u_1,FErr_MIPS_24_l_1,Bkg_MIPS_24_1,Sig_conf_MIPS_24_1,Rhat_MIPS_24_1,n_eff_MIPS_24_1,Pval_res_24_1,flag_mips_24_1,XID+_rerun_mips,Source_Name_2,DEC_2,E_RA_fir,E_DEC_fir,Total_flux_fir,E_Total_flux_fir,Peak_flux_fir,E_Peak_flux_fir,S_Code_fir,Maj_fir,Min_fir,PA_fir,E_Maj_fir,E_Min_fir,E_PA_fir,DC_Maj_fir,DC_Min_fir,DC_PA_fir,FLAG_WORKFLOW_fir,Prefilter_fir,NoID_fir,lr_fin_fir,optRA_fir,optDec_fir,LGZ_Size_fir,LGZ_Width_fir,LGZ_PA_fir,Assoc_fir,Assoc_Qual_fir,Art_prob_fir,Blend_prob_fir,Hostbroken_prob_fir,Imagemissing_prob_fir,Zoom_prob_fir,Created_fir,Position_from_fir,Renamed_from_fir,FLAG_OVERLAP_RADIO_fir,flag_clean_radio_fir,NUMBER_fir,X_IMAGE_fir,Y_IMAGE_fir,ALPHA_J2000_fir,DELTA_J2000_fir,flag_clean_fir,NUMBER_OPTICAL_fir,NUMBER_SPITZER_fir,FLAG_OVERLAP_fir,Separation_fir,help_id_fir,RA_HELP_fir,DEC_HELP

In [45]:
source_name_keep = fir_v3['Source_Name']
mask = [(name in source_name_keep) for name in lofar_fir['Source_Name']]

In [48]:
Table.write(lofar_fir[mask],'data/data_release/XID+_lofar_ELAIS-N1.fits',overwrite=True)

In [42]:
tmp = SkyCoord(fir_v1['optRA_v1'][0],fir_v1['optDec_v1'][0],unit='deg')

In [43]:
tmp.ra.hms

hms_tuple(h=15.0, m=59.0, s=58.63031751742497)